# **Problems**

In [254]:
# Imports
import numpy as np

## **Problem 1: Generating Random Boolean Functions**

The [Deutsch–Jozsa algorithm](https://quantum.cloud.ibm.com/learning/en/modules/computer-science/deutsch-jozsa) is designed to work with functions that accept a fixed number of [Boolean inputs](https://realpython.com/python-boolean/) and return a single [Boolean output](https://realpython.com/python-boolean/).
Each function is guaranteed to be either constant (always returns `False` or always returns `True`) or balanced (returns `True` for exactly half of the possible input combinations).
Write a Python function `random_constant_balanced` that returns a randomly chosen function from the set of constant or balanced functions taking four Boolean arguments as inputs.

### `random_constant()` Function

This function generates a **constant function** that always returns the same value (0 or 1) regardless of the input.

- **Input:** None (generates randomly)
- **Output:** A function that takes 4 boolean arguments (a, b, c, d) and always returns 0 or 1
- **Key Property:** All 16 possible inputs return the same value

In [255]:
# Constant Function Generator
def random_constant():
    """
    Returns a constant function that always returns the same value (0 or 1)
    regardless of the 4 boolean inputs (a, b, c, d)
    """
    # Randomly pick 0 or 1
    constant_value = np.random.choice([0, 1])
    return lambda a, b, c, d: constant_value

### `random_balanced()` Function

This function generates a **balanced function** that returns 1 for exactly half of the 16 possible inputs and 0 for the other half.

- **Input:** None (generates randomly)
- **Output:** A function that takes 4 boolean arguments (a, b, c, d)
- **Key Property:** Returns 1 for exactly 8 out of 16 inputs, and 0 for the other 8
- **Method:** Creates a random mapping by shuffling 8 ones and 8 zeros across all possible input combinations

In [256]:
# Balanced Function Generator
def random_balanced():
    """
    Returns a balanced function that returns 1 for exactly 8 out of 16 inputs
    and 0 for the other 8 inputs
    """
    # Create array of 8 ones and 8 zeros, then shuffle randomly
    outputs = np.random.permutation([1] * 8 + [0] * 8)
    
    # Create a mapping from each input combination to an output
    # For each number from 0 to 15, extract its 4 bits as (a, b, c, d)
    input_map = {}
    for i in range(16):
        # Extract each bit: bit 3, 2, 1, 0
        a = (i >> 3) & 1
        b = (i >> 2) & 1
        c = (i >> 1) & 1
        d = i & 1
        input_map[(a, b, c, d)] = outputs[i]

    print(input_map)

    return lambda a, b, c, d: input_map[(a, b, c, d)]

### `random_constant_balanced()` Function

This is the **main function** that randomly chooses between creating a constant or balanced function.

- **Input:** None (generates randomly)
- **Output:** A function that is either constant or balanced (randomly chosen)
- **Purpose:** Provides test functions for the Deutsch-Jozsa algorithm to classify
- **Usage:** Call this function to get a random test case, then use the Deutsch-Jozsa algorithm to determine if it's constant or balanced

In [257]:
# Combined Function (randomly picks constant or balanced)
def random_constant_balanced():
    """
    Returns a randomly chosen function that is either:
    - Constant: always returns 0 or always returns 1
    - Balanced: returns 1 for exactly half (8 out of 16) input combinations
    """
    is_constant = np.random.choice([True, False])
    
    if is_constant:
        return random_constant()
    else:
        return random_balanced()

In [258]:
# Testing the function
f = random_constant_balanced()

# Test with some inputs
print("Testing the randomly generated function with different inputs:")
test_inputs = [
    (0, 0, 0, 0),
    (0, 0, 0, 1),
    (1, 1, 1, 1),
    (1, 0, 1, 0),
]

for inputs in test_inputs:
    result = f(*inputs)
    print(f"f{inputs} = {result}")

{(0, 0, 0, 0): np.int64(0), (0, 0, 0, 1): np.int64(1), (0, 0, 1, 0): np.int64(0), (0, 0, 1, 1): np.int64(0), (0, 1, 0, 0): np.int64(0), (0, 1, 0, 1): np.int64(1), (0, 1, 1, 0): np.int64(0), (0, 1, 1, 1): np.int64(0), (1, 0, 0, 0): np.int64(1), (1, 0, 0, 1): np.int64(1), (1, 0, 1, 0): np.int64(1), (1, 0, 1, 1): np.int64(1), (1, 1, 0, 0): np.int64(1), (1, 1, 0, 1): np.int64(0), (1, 1, 1, 0): np.int64(1), (1, 1, 1, 1): np.int64(0)}
Testing the randomly generated function with different inputs:
f(0, 0, 0, 0) = 0
f(0, 0, 0, 1) = 1
f(1, 1, 1, 1) = 0
f(1, 0, 1, 0) = 1


## **Problem 2: Classical Testing for Function Type**

[Deutsch's algorithm](https://quantum.cloud.ibm.com/learning/en/courses/fundamentals-of-quantum-algorithms/quantum-query-algorithms/deutsch-algorithm) is designed to demonstrate a [potential advantage of quantum computing](https://www.quantamagazine.org/john-preskill-explains-quantum-supremacy-20191002/) over classical computation.
To understand this advantage, we must first understand the classical cost of solving the underlying problem.
Write a Python function `determine_constant_balanced` that takes as input a function `f`, as defined in Problem 1.
The function should analyze `f` and return the string `"constant"` or `"balanced"` depending on whether the function is constant or balanced.
Write a brief note on the efficiency of your solution.
What is the maximum number of times you need to call `f` to be 100% certain whether it is constant or balanced?


In [259]:
def determine_constant_balanced(f):
    """
    Classically determine if a function f: {0,1}^4 -> {0,1} is constant or balanced.

    Args:
        f: A function that takes 4 boolean arguments and returns 0 or 1
    
    Returns:
        str: "constant" or "balanced"

    Note:
        Requires 16 function calls to be 100% certain of the result.
        This is the classical cost of solving this classification problem.
    """
    # Collect outputs for all 16 possible inputs
    outputs = []
    for i in range(16):
        a = (i >> 3) & 1
        b = (i >> 2) & 1
        c = (i >> 1) & 1
        d = i & 1
        outputs.append(f(a, b, c, d))
    print(outputs)
    
    # Count the number of 1s
    count_ones = sum(outputs)
    
    # Constant: all outputs are the same (0 ones or 16 ones)
    # Balanced: exactly half are 1s (8 ones)
    if count_ones == 0 or count_ones == 16:
        return "constant"
    else:
        return "balanced"

In [260]:
# Test with your random_constant_balanced function
f = random_constant_balanced()
print(determine_constant_balanced(f))

[np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1)]
constant


## **Problem 3: Quantum Oracles**

In [261]:
# Problem 3

## **Problem 4: Deutsch's Algorithm with Qiskit**

In [262]:
# Problem 4

## **Problem 5: Scaling to the Deutsch–Jozsa Algorithm**

In [263]:
# Problem 5

## not imprortance stuff for now

In [264]:
# Deterministic operations
def f1(a):
    return 0

def f2(a):
    if a == 0:
        return 0
    return 1

def f3(a):
    if a == 0:
        return 1
    return 0

def f4(a):
    return 1

In [ ]:
# example usage of the random_constant_balanced function
f = random_constant_balanced()
result = f(0, 0, 0, 1)
print(result)